In [29]:
import os
import sys
import copy
import json
import math
import pyproj
import numpy as np
import pandas as pd
import scipy.spatial
import networkx as nx
import geopandas as gpd
import matplotlib.pyplot as plt

from tqdm import tqdm
from functools import partial
from shapely.ops import transform
from shapely.geometry import shape, mapping, Point, LineString
from dijkstar import Graph, find_path

# Import data

In [2]:
# Choose origin and destination
origin = 'ward'
destination = 'borough'
size = 'small'
if size == 'small':
    size += '_'

In [5]:
# Import coordinates to get names and ids of wards and boroughs
vertices = gpd.read_file(f'../data/input/misc/london_entire_small_graph_vertices.geojson')
triangulated_polygons = gpd.read_file(f'../data/input/misc/london_entire_{size}network_vertex_triangulation.geojson')

# Populate edges

In [7]:
edge_rows = []
vertex_columns = vertices.columns.values

project = partial(
    pyproj.transform,
    pyproj.Proj('EPSG:4326'),
    pyproj.Proj('EPSG:32633'))

for index,te in tqdm(triangulated_polygons.iterrows(),total=triangulated_polygons.shape[0]):

#     print(te)
    v1id = int(te['POINTA'])+1
    v2id = int(te['POINTB'])+1
    v3id = int(te['POINTC'])+1
    
#     if v1id > vertices.shape[0] or v2id > vertices.shape[0] or v3id > vertices.shape[0]:
#         print('MAX',vertices.shape[0])
#         print('v1id',v1id)
#         print('v2id',v2id)
#         print('v3id',v3id)
        
#     if v1id == 0 or v2id == 0 or v3id == 0:
#         print('IDS start from 0')
#         print('v1id',v1id)
#         print('v2id',v2id)
#         print('v3id',v3id)
    
    v1 = vertices[vertices['POINT_NUM']==v1id]
    v2 = vertices[vertices['POINT_NUM']==v2id]
    v3 = vertices[vertices['POINT_NUM']==v3id]
     
    try:
        
        # Create 3 rows for each triangulated polygon
        geometry1 = LineString([shape(v1['geometry'].values[0]),shape(v2['geometry'].values[0])])
#         geometry1_reprojected = transform(project, geometry1)
#         distance1 = geometry1_reprojected.length
        
        row1 = {'ID1':v1['ID'].values[0],'ROAD_ID1':v1['ROAD_ID'].values[0],'EDGE_ID1':v1['EDGE_ID'].values[0],'TYPE1':v1['TYPE'].values[0],#'DISTANCE':distance1,
                'ID2':v2['ID'].values[0],'ROAD_ID2':v2['ROAD_ID'].values[0],'EDGE_ID2':v2['EDGE_ID'].values[0],'TYPE2':v2['TYPE'].values[0],'GEOMETRY':geometry1}

        geometry2 = LineString([shape(v1['geometry'].values[0]),shape(v3['geometry'].values[0])])
#         geometry2_reprojected = transform(project, geometry2)
#         distance2 = geometry2_reprojected.length
        
        row2 = {'ID1':v1['ID'].values[0],'ROAD_ID1':v1['ROAD_ID'].values[0],'EDGE_ID1':v1['EDGE_ID'].values[0],'TYPE1':v1['TYPE'].values[0],#'DISTANCE':distance2,
                'ID2':v3['ID'].values[0],'ROAD_ID2':v3['ROAD_ID'].values[0],'EDGE_ID2':v3['EDGE_ID'].values[0],'TYPE2':v3['TYPE'].values[0],'GEOMETRY':geometry2}
        
        geometry3 = LineString([shape(v2['geometry'].values[0]),shape(v3['geometry'].values[0])])
#         geometry3_reprojected = transform(project, geometry3)
#         distance3 = geometry3_reprojected.length
        
        row3 = {'ID1':v2['ID'].values[0],'ROAD_ID1':v2['ROAD_ID'].values[0],'EDGE_ID1':v2['EDGE_ID'].values[0],'TYPE1':v2['TYPE'].values[0],#'DISTANCE':distance3,
                'ID2':v3['ID'].values[0],'ROAD_ID2':v3['ROAD_ID'].values[0],'EDGE_ID2':v3['EDGE_ID'].values[0],'TYPE2':v3['TYPE'].values[0],'GEOMETRY':geometry3}
    except Exception:
        print('-------- V1 --------')
        print(v1)
        print()
        print('-------- V2 --------')
        print(v2)
        print()
        print('-------- V3 --------')
        print(v3)
    
    # Append rows
    edge_rows.append(row1)
    edge_rows.append(row2)
    edge_rows.append(row3)

# Pass rows to geodataframe
edges_df = pd.DataFrame.from_dict(edge_rows)
edges_gdf = gpd.GeoDataFrame(edges_df, crs="EPSG:4326", geometry='GEOMETRY')

100%|██████████| 36831/36831 [03:45<00:00, 163.29it/s]


In [16]:
# Reproject edges to get distance in meters
edges_gdf_copy = copy.deepcopy(edges_gdf)
edges_gdf_copy = edges_gdf_copy.to_crs("EPSG:32633")
edges_gdf_copy['DISTANCE'] = edges_gdf_copy['GEOMETRY'].apply(lambda x: int(x.length))
# Merge edge gdf with its reprojection to get distance in meters
edges_gdf_final = pd.merge(edges_gdf,edges_gdf_copy[['ID1','ID2','DISTANCE']],how='left',on=['ID1','ID2'])

# Export triangulated edges

In [27]:
edges_gdf_final.head(1)

,ID1,ROAD_ID1,EDGE_ID1,TYPE1,ID2,ROAD_ID2,EDGE_ID2,TYPE2,GEOMETRY,DISTANCE
0,O167V65WKCQTW2112_0,O167V65WKCQTW,O167V65WKCQTW2112,None,O167V9HWKBDAW2119_0,O167V9HWKBDAW,O167V9HWKBDAW2119,None,"LINESTRING (-0.10370 51.29423, -0.10956 51.29633)",476


In [28]:
# Export to csv
edges_gdf_final.to_file(f'../data/input/misc/london_{size}network_triangulated_edges.geojson',driver='GeoJSON')